# Code for the paper

## Listing 1

In [ ]:
import nimare
sl_dset = nimare.io.convert_sleuth_to_dataset("data/sleuth_file.txt")

## Listing 2

In [ ]:
nimare.extract.fetch_neurosynth("data/", unpack=True)
ns_dset = nimare.io.convert_neurosynth_to_dataset(
    "data/database.txt",
    "data/features.txt"
)

## Listing 3

In [ ]:
from nimare.meta import kernel

mkda_kernel = kernel.MKDAKernel(r=10)
mkda_ma_maps = mkda_kernel.transform(sl_dset, return_type="image")
kda_kernel = kernel.KDAKernel(r=10)
kda_ma_maps = kda_kernel.transform(sl_dset, return_type="image")
ale_kernel = kernel.ALEKernel(sample_size=20)
ale_ma_maps = ale_kernel.transform(sl_dset, return_type="image")

### Figure 3

## Listing 4

In [ ]:
from nimare.meta.cbma import mkda

mkdad_meta = mkda.MKDADensity()
mkdad_results = mkdad_meta.fit(sl_dset)

## Listing 5

In [ ]:
from nimare.meta.cbma import ale

ijk = ns_dset.coordinates[["i", "j", "k"]].values
meta = ale.SCALE(n_iters=2500, ijk=ijk, kernel__n=20)
scale_results = meta.fit(sl_dset)

## Listing 6

In [ ]:
from nimare.meta.cbma import mkda

dset_first10 = sl_dset.slice(dset.ids[:10])
dset_sec10 = sl_dset.slice(ns_dset.ids[10:20])
meta = mkda.MKDAChi2()
mkdac_results = meta.fit(dset_first10, dset_sec10)

### Figure 4

In [ ]:
# Additional meta-analyses for figures
meta = mkda.KDA()
kda_results = meta.fit(sl_dset)

meta = ale.ALE()
ale_results = meta.fit(sl_dset)

# Meta-analytic maps across estimators
results = [mkdad_results, mkdac_results, kda_results, ale_results, scale_results]
for r in results:
    pass

## Listing 7

In [ ]:
from nimare.meta import ibma

meta = ibma.DerSimonianLaird()
dsl_results = meta.fit(img_dset)

### Figure 5

In [ ]:
# Additional meta-analyses for figures
meta = ibma.Stouffers(use_sample_size=False)
stouffers_results = meta.fit(img_dset)

meta = ibma.Stouffers(use_sample_size=True)
weighted_stouffers_results = meta.fit(img_dset)

meta = ibma.Fishers()
fishers_results = meta.fit(img_dset)

meta = ibma.PermutedOLS()
ols_results = meta.fit(img_dset)

meta = ibma.WeightedLeastSquares()
wls_results = meta.fit(img_dset)

meta = ibma.Hedges()
hedges_results = meta.fit(img_dset)

# Use atlas for likelihood-based estimators
from nilearn import datasets
atlas = datasets.fetch_atlas_yeo_2011()
masker = NiftiLabelsMasker(atlas["thick_17"])

meta = ibma.VarianceBasedLikelihood(method="reml", mask=masker)
vbl_results = meta.fit(img_dset)

meta = ibma.SampleSizeBasedLikelihood(method="reml", mask=masker)
ssbl_results = meta.fit(img_dset)

# Plot statistical maps from IBMAs
results = [
    dsl_results, 
    stouffers_results, 
    weighted_stouffers_results, 
    fishers_results, 
    ols_results, 
    wls_results, 
    hedges_results, 
    vbl_results, 
    ssbl_results
]
for r in results:
    pass

## Listing 8

In [ ]:
from nimare.correct import FWECorrector

mc_corrector = FWECorrector(method="montecarlo", n_iters=10000, n_cores=1)
mc_results = mc_corrector.transform(mkdad_meta.results)

b_corrector = FWECorrector(method="bonferroni")
b_results = b_corrector.transform(mkdad_meta.results)

### Figure 6

## Listing 9

In [ ]:
kernel = kernel.ALEKernel(n=20)
meta = ale.ALESubtraction(kernel_transformer=kernel, n_iters=10000)
subtraction_results = meta.fit(dset1, dset2)

### Figure 7

## Listing 10

In [ ]:
amygdala_ids = ns_dset.get_studies_by_mask("amygdala_roi.nii.gz")
dset_amygdala = ns_dset.slice(amygdala_ids)

sphere_ids = ns_dset.get_studies_by_coordinate([[20, 20, 20]], r=6)
dset_sphere = ns_dset.slice(sphere_ids)

## Listing 11

In [ ]:
meta_amyg = ale.ALE()
results_amyg = meta_amyg.fit(dset_amygdala)

meta_sphere = ale.ALE()
results_sphere = meta_sphere.fit(dset_amygdala)

### Figure 8

## Listing 12

### Figure 9

## Listing 13

In [ ]:
dset_with_abstracts = nimare.extract.download_abstracts(
    dset, 
    email="example@email.com"
)

## Listing 14

In [ ]:
model = nimare.annotate.lda.LDAModel(
    dset_with_abstracts.texts, text_column="abstract", 
    n_topics=100, n_iters=10000)
model.fit()

### Table 1

In [ ]:
model.p_word_g_topic_

## Listing 15

In [ ]:
dset_first1000 = dset_with_abstracts.slice(
    dset_with_abstracts.ids[:1000])
counts_df = nimare.annotate.text.generate_counts(
    dset_first1000.texts, text_column="abstract", 
    tfidf=False, min_df=1, max_df=1.)
model = nimare.annotate.gclda.GCLDAModel(
    counts_df, dset_first1000.coordinates, 
    n_regions=2, n_topics=100, symmetric=2,
    mask=dset.masker.mask_img)
model.fit(n_iters=10000)

### Table 2

In [ ]:
model.p_word_g_topic_

### Figure 10

In [ ]:
from nilearn import image

topic_img_4d = model.masker.unmask(model.p_voxel_g_topic_)
for i in range(5):
    topic_img = image.index_img(topic_img_4d, index=i)
    plot_stat_map(topic_img)

## Listing 16

In [ ]:
from nimare.decode.continuous import CorrelationDecoder

decoder = CorrelationDecoder(feature_group="Neurosynth_tfidf",
                             frequency_threshold=0.001,
                             meta_estimator=mkda.MKDAChi2,
                             target_img="z_desc-specificity")
decoder.fit(ns_dset)
decoding_results = decoder.transform("pain_map.nii.gz")

### Figure 11

In [ ]:
plot_stat_map("pain_map.nii.gz")

### Table 3

In [ ]:
decoding_results

## Listing 17

In [ ]:
from nimare.decode.continuous import CorrelationDistributionDecoder

decoder = CorrelationDistributionDecoder(
    feature_group="Neurosynth_tfidf",
    frequency_threshold=0.001,
    target_img="z")
decoder.fit(ns_dset)
decoding_results = decoder.transform("pain_map.nii.gz")

### Table 4

In [ ]:
decoding_results

## Listing 18

In [ ]:
from nimare.decode.discrete import BrainMapDecoder

decoder = BrainMapDecoder(feature_group="Neurosynth_tfidf",
                          frequency_threshold=0.001,
                          u=0.05, correction="fdr_bh")
decoder.fit(ns_dset)
decoding_results = decoder.transform(amygdala_ids)

### Table 5

In [ ]:
decoding_results.sort_values(by="probReverse", ascending=False).head()

## Listing 19

In [ ]:
from nimare.decode.discrete import NeurosynthDecoder

decoder = NeurosynthDecoder(feature_group="Neurosynth_tfidf",
                            frequency_threshold=0.001,
                            u=0.05, correction="fdr_bh")
decoder.fit(ns_dset)
decoding_results = decoder.transform(amygdala_ids)

### Table 6

In [ ]:
decoding_results.sort_values(by="probReverse", ascending=False).head()